# Metamorpher controler 
This python script connects jupyther to Metamorph and allows the user to control the microscope through python scripts instead of journals.
\
**For the user: it is important to run (ctr+enter) all the following cells (connecting MetaMorph, Other useful tools) before launching an acquisition - they should print out "Good to go"**

### Connecting MetaMorph
Import the clr library and the dll library
\
Add the path to the acquisition journal and the folder were the temporary images are store
\
**=> Run the following cell (ctr+enter or Run/Executer)**

In [1]:
import clr
clr.AddReference('D:\\Users\\Iris\\Interop.MMAppLib.dll')
import MMAppLib
core=MMAppLib.UserCallClass()
#print(dir(core))
jnl_path='C:\\MM\\app\\mmproc\\journals\\s.JNL' # path to the journal taking a snapshot and saving it somewhere
path_tmp='C:\\TEMP\\' #location of the temporary image file generated during acquisition - can be modified in MM
print("Good to go")

FileNotFoundException: Unable to find assembly 'D:\Users\Iris\Interop.MMAppLib.dll'.
   at Python.Runtime.CLRModule.AddReference(String name)

### Other useful tools: libraries and functions

Import other useful libraries

In [13]:
# libraries 
import matplotlib.pyplot as plt
import numpy as np
import math 
import time
import tifffile as tif
from PIL import Image
import pandas as pd
import json
from PIL.TiffTags import TAGS
import PIL
from scipy.optimize import curve_fit

print("Libraries are Good to go")

Libraries are Good to go


Tool functions:
- set_pos: set the position of the microscope from predetermined coordinates 
- add_pos: add current position to a dataframe 
- set_wl: select the wavelenght channel 
- set_exposure: set the exposure (ms) 
- get_exposure: get the value of current exposure
- set_gain: set the gain
- set_laser: set the intensities of the lasers 
- acquire: acquire an image and retrun it as a matrix of intensity values - can be used as an illumination function for activation
- save_meta: acquire 2 files: the image as a .tif file and the meta data as a .txt file
+ other uselful tool functions to deal with matrices 


**=> Run the following cell (ctr+enter or Run/Executer)**

In [5]:
#coord=[x,y] in um
def set_pos(coord):
    core.SetMMVariable('Device.Stage.XPosition',coord[0])
    core.SetMMVariable('Device.Stage.YPosition',coord[1])
    core.SetMMVariable('Device.Focus.CurPos',coord[2])
    return 

#add the current position to a dataframe
def add_pos(all_pos):  
    a,b,x=core.GetMMVariable('Device.Stage.XPosition',0)
    a,b,y=core.GetMMVariable('Device.Stage.YPosition',0)
    a,b,z=core.GetMMVariable('Device.Stage.CurPos',0)
    current_pos=pd.DataFrame(np.array([[x,y,z]]),columns=['x', 'y', 'z']) 
    all_pos_updated=all_pos.append(current_pos)    
    return all_pos_updated


#set the wavelenght (using a predefined channel)
def set_wl(chan):
    core.SetMMVariable('Device.Illumination.Setting',chan) 
    return 
        
#set exposure (number in ms)
def set_exposure(exp):
    core.SetMMVariable('Device.Illumination.Exposure',exp)
    return 

def get_exposure():
    a,b,exp=core.GetMMVariable('Device.Illumination.Exposure',0)
    return exp

#set the gain  
def set_gain(g): 
    core.SetMMVariable('Device.Illumination.EMGain',g)
    return 

def get_gain():
    g=core.GetMMVariable('Device.Illumination.EMGain',0)
    return g
def set_laser(laser_405,laser_488,laser_561,laser_642):
    core.SetMMVariable('Component.Laser_DAQ_Intensity_405.Position',laser_405)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_488.Position',laser_488)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_561.Position',laser_561)
    core.SetMMVariable('Component.Laser_DAQ_Intensity_642.Position',laser_642)
    
#image acquisition
def acquire(jnl_path,path_tmp):
    #run journal doing the acquisition
    core.RunJournal(jnl_path)
    #open the open image as tif
    pixvals=np.array((Image.open(path_tmp+'temp.tif')))
    return pixvals

#image acquisition + save as .tif
def acquire_save(name,jnl_path,path_tmp):
    #run journal doing the acquisition
    core.RunJournal(jnl_path)
    #open the temporary image saved as a .tif
    pixvals=np.array((Image.open(path_tmp+'temp.tif')))
    #save it as a definitive .tif file in the user's file location 
    tif.imwrite(name,pixvals,append=True)
    
#because there is an issue with the coordinate formating when doing loops 
def mat_conversion(mat): 
    x=float(mat[0])
    y=float(mat[1])
    z=float(mat[2])
    return [x,y,z]

#convert the values of a dataframe to a list of positions
def dataframe_to_coordinates(df):
    coord_m=[]
    for n in range(df.shape[0]):
        coord_n=[]
        for element in df.iloc[n]:
            coord_n.append(element)
        coord_m.append(coord_n)
    return coord_m

# normalize a matrix by its highest value 
def normalization(matrix):
    N,M=np.shape(matrix)
    new_mat=np.zeros([N,M])
    mat_max=np.amax(matrix)
    for i in range(N):
        for j in range(M):
            new_mat[i][j]=matrix[i][j]/mat_max            
    return new_mat

print("Functions are Good to go")

Functions are Good to go


Functions for auto-focus:
- take 3 to 5 pictures
- determine the Brenner gradient for each 
- fit Brenner=f(z) to a polynomial function
- the focus corresponds to the max of the polynomial function

The Brenner gradient defined as $B=\sum_{i=1}^{N}\sum_{j=1}^{M} (s_{i,j} - s_{i+m,j})^2$ 
\
with N,M the dimention of the picture, s the pixel value.
This gradient measure the difference between a pixel and its neighbor (here, at a distance m).
\
The pixel values are normalized before computation of the Brenner gradient

In [6]:
# determine the brenner gradient of a matrix by comparing neighbors at a distance m 
def brenner(matrix,m):
    N,M=np.shape(matrix)
    B=0
    for i in range(N-m):
        for j in range(M):
            s=matrix[i][j]
            n=matrix[i+m][j]
            B+=(s*n)**2
    return B

#fitting function
def f_brenner(z,a,b,z0):
    return a/(b+(z-z0)**4)

#auto focus
def autofocus(m):
    a,b,x=core.GetMMVariable('Device.Stage.XPosition',0)
    a,b,y=core.GetMMVariable('Device.Stage.YPosition',0)
    a,b,z=core.GetMMVariable('Device.Stage.CurPos',0)
    list_z=[z-1,z,z+1]
    list_brenner=[]
    for z_AF in list_z:
        set_wl('CSUTRANS')
        coord=[x,y,z_AF]
        set_pos(coord)
        image=acquire(jnl_path,path_tmp)
        image=normalization(image)
        B=brenner(image,m)
        list_brenner.append(B)
    popt,pcov=curve_fit(f_brenner,list_z,list_brenner)
    a,b,z0=popt
    return z0

print("Autofocus is Good to go")

Autofocus is Good to go


# Acquisition
currently without auto focus
\
**=> Don't forget to run the cells to set the parameters (ctr+enter or Run/Executer)**

Path to folder

In [5]:
PATH = 'D:\\Users\\Iris\\test python\\'

Name of the cell and plasmids (to set up the name of the files) 

In [6]:
#cell: HT or Kras
#plasmid: optopatch or ERKKTR
cell='HT'
plasmid='ERKKTR'
iteration=0

Lasers settings

In [7]:
#laser intensity: 
laser_405=0
laser_488=40
laser_561=16
laser_642=0

#observations and activation channels 
#channel 'CSUTRANS', 'CSU405','CSU488','CSU561','CSU640'
wl_obs=['CSUTRANS','CSU561']
wl_act=['CSU488']

#exposure (ms)
exp=100
#gain 
G=400

Time lapse settings

In [8]:
'''Observation - acquisition '''
#time step (s)
step=6
# total time of acquisition (s)
t_final=60

'''Activations pulses'''
#activation
act_pulse=1 # duration of 1 activation pulse
act_lenght=20 #duration of total time of activation
act_start=20 #start time of activation

Description 

In [9]:
#description (text)
D=str(laser_488)

Run to acquire current position
\
**=> Running the following cell (invisible) will create a new position folder (ctr+enter or Run/Executer)**

In [10]:
#file name 
name='current_position'
name=PATH+name+'.csv'
pos_experiment=pd.DataFrame(np.array([[0,0,0]]),columns=['x', 'y', 'z'])
pos_experiment.drop([0], inplace = True)
print("new file created and Good to go") 

NameError: name 'PATH' is not defined

In [12]:
# to run to add a new position
pos_experiment=add_pos(pos_experiment)
pos_experiment.to_csv(name, index = False)

Summary of parameters
\
**=> Run the following cell to print out the parameters (ctr+enter or Run/Executer)**

In [11]:
# path to folder \
#PATH = 'D:\\Users\Iris\observation_290322\\'+cell+'_'+plasmid+'\\laser power\\'+str(laser)+'\\'
#individual file name 
name=cell+'_'+plasmid+'_'+str(laser_488)+'_'+str(iteration)+'_'
name=PATH+name
print('File name: ',name)

print('total acquisition duration: ',t_final,'s - time steps: ',step,'s')

t_activation=[act_start+t for t in range(0,act_lenght,act_pulse)]
if wl_act:
    print('activation steps: ',t_activation)
print('observation wavelenght: ',wl_obs,'- activation wavelenght: ',wl_act,)
print('Lasers intensities: 405=',laser_405,' - 488=',laser_488,' - 561=',laser_561,' - 642=', laser_642)
print('Exposure: ',exp)
print('Gain: ',G)
#coordinates ([x,y,z]) - from file 
file_name_csv='current_position'
pos_csv=pd.read_csv(PATH+file_name_csv+'.csv')
coord_m=dataframe_to_coordinates(pos_csv)
print('coordinates:')
display(coord_m)



NameError: name 'cell' is not defined

Acquisition
\
**=> Run the following cell to launch the acquisition (ctr+enter or Run/Executer)**

In [14]:
'''script'''
t0=round(time.time())
time_points_obs=np.array([round(t0+t) for t in range(0,t_final+1,step)])
time_points_act=[round(t0+t) for t in t_activation]
local_time=round(time.time())
counter=0
while counter<len(time_points_obs):
    for point in time_points_obs:
        while local_time<point:
            local_time=round(time.time())
            time.sleep(0.5)
        #set up the microscope:
        set_laser(laser_405,laser_488,laser_561,laser_642)
        set_exposure(exp)
        set_gain(G)
        if local_time in time_points_act:
            if wl_act: #check that an activation has been set up 
                set_wl(wl_act[0])
                acquire(jnl_path,path_tmp)
                print('=> => => act',local_time-t0)
        for n in range(len(coord_m)):
            coord=mat_conversion(coord_m[n])
            set_pos(coord)
            for m in range(len(wl_obs)):
                wl=wl_obs[m]
                set_wl(wl)
                T=local_time-t0
                print('obs',T)
                # save meta data for the acquisition
                coord_str='x='+str(coord[0])+', y='+str(coord[1])+', z='+str(coord[2])
                if wl_act:
                    MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl,'Activation Channel: '+wl_act[0],'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642),'Description: '+str(D)])
                else:
                    MD=np.array(['Position: '+coord_str,'Acquisition Channel: '+wl,'Exposure: '+str(exp),'Gain: '+str(G),'Laser Intensity 405: '+str(laser_405),'Laser Intensity 488: '+str(laser_488),'Laser Intensity 561: '+str(laser_561),'Laser Intensity 642: '+str(laser_642),'Description: '+str(D)])
                MD_name=name+'pos'+str(n)+'_wl'+str(wl)+'_metadata.txt'
                with open(MD_name, 'w') as f:
                    for line in MD:
                        f.write(line)
                        f.write('\n')
                #save individual image
                name_n=name+'pos'+str(n)+'_'+'wl'+str(wl)+'_'+str(T)
                acquire_save(name_n,jnl_path,path_tmp)
                # save timelapse as stack 
                stack_name_n=name+'pos'+str(n)+'_'+'wl'+str(wl)+'.tif'
                acquire_save(stack_name_n,jnl_path,path_tmp)
                counter+=1

obs 0
obs 0
obs 0
obs 0
obs 7
obs 7
obs 7
obs 7
obs 15
obs 15
obs 15
obs 15
=> => => act 22
obs 22
obs 22
obs 22
obs 22
=> => => act 30
obs 30
obs 30
obs 30
obs 30
=> => => act 30
obs 30
obs 30
obs 30
obs 30
obs 47
obs 47
obs 47
obs 47
obs 47
obs 47
obs 47
obs 47
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
obs 62
